# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [1]:
# YOUR CHANGES HERE

import pandas as pd
from pathlib import Path

# Load the price series (expects columns: month, price)
def _load_prices(path="strawberry-prices.tsv"):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError("strawberry-prices.tsv not found next to this notebook.")
    df = pd.read_csv(p, sep="\t")
    if "month" not in df.columns or "price" not in df.columns:
        raise ValueError("Expected columns 'month' and 'price' in strawberry-prices.tsv")
    df["month"] = pd.to_datetime(df["month"]).dt.to_period("M")
    df = df.sort_values("month").reset_index(drop=True)
    df["price"] = df["price"].astype(float)
    return df

df = _load_prices()

# Split training (2020–2023) and test year (2024)
train = df[(df["month"] >= "2020-01") & (df["month"] <= "2023-12")].copy()
test_2024 = df[(df["month"] >= "2024-01") & (df["month"] <= "2024-12")].copy()

# Seasonal monthly averages on training data
train["moy"] = train["month"].dt.month
seasonal_avg = train.groupby("moy")["price"].mean()

# Predict 2024 by month-of-year
pred_2024 = test_2024.copy()
pred_2024["moy"] = pred_2024["month"].dt.month
pred_2024["price"] = pred_2024["moy"].map(seasonal_avg)

# Save required backtest file
out_backtest = pred_2024[["month","price"]].copy()
out_backtest["month"] = out_backtest["month"].astype(str)  # YYYY-MM
out_backtest.to_csv("strawberry-backtest.tsv", sep="\t", index=False)
print("Wrote strawberry-backtest.tsv")


Wrote strawberry-backtest.tsv


Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [2]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

# Load actual and predicted 2024
actual_2024 = df[(df["month"] >= "2024-01") & (df["month"] <= "2024-12")][["month","price"]] \
                .rename(columns={"price":"actual"}).copy()

pred_2024 = pd.read_csv("strawberry-backtest.tsv", sep="\t")
pred_2024["month"] = pd.to_datetime(pred_2024["month"]).dt.to_period("M")
pred_2024 = pred_2024.rename(columns={"price":"pred"})

# Join and compute errors
eval_2024 = actual_2024.merge(pred_2024, on="month", how="inner")
eval_2024["error"] = eval_2024["actual"] - eval_2024["pred"]  # positive => underpredicted

err_mean = float(eval_2024["error"].mean())
err_std  = float(eval_2024["error"].std(ddof=1))

pd.DataFrame([{"mean_error": err_mean, "std_error": err_std}]) \
  .to_csv("backtest-errors.tsv", sep="\t", index=False)

print(f"Mean error: {err_mean:.4f} | Std dev: {err_std:.4f}")


Mean error: -0.0049 | Std dev: 0.2746


Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [3]:
# YOUR CHANGES HERE

import pandas as pd

df_all = df.copy()
df_all["moy"] = df_all["month"].dt.month

# Seasonal average over 2020–2024
seasonal_avg_25 = df_all[(df_all["month"] >= "2020-01") & (df_all["month"] <= "2024-12")] \
                     .groupby("moy")["price"].mean()

# Construct 2025 months and map seasonal average
months_2025 = pd.period_range("2025-01", "2025-12", freq="M")
forecast_2025 = pd.DataFrame({"month": months_2025})
forecast_2025["moy"] = forecast_2025["month"].dt.month
forecast_2025["price"] = forecast_2025["moy"].map(seasonal_avg_25)

out_forecast = forecast_2025[["month","price"]].copy()
out_forecast["month"] = out_forecast["month"].astype(str)  # YYYY-MM
out_forecast.to_csv("strawberry-forecast.tsv", sep="\t", index=False)
print("Wrote strawberry-forecast.tsv")


Wrote strawberry-forecast.tsv


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [4]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

BUDGET = 1_000_000.00   # dollars
FREEZE_COST = 0.20      # $/pint
DISCOUNT = 0.10         # 10% off sell price for frozen strawberries

forecast = pd.read_csv("strawberry-forecast.tsv", sep="\t")
forecast["month"] = pd.to_datetime(forecast["month"]).dt.to_period("M")
forecast = forecast.sort_values("month").reset_index(drop=True)

best = None
rows = []

for i, buy_row in forecast.iterrows():
    for j, sell_row in forecast.iterrows():
        if j <= i:
            continue  # must sell after buy

        buy_price = float(buy_row["price"])
        sell_price = float(sell_row["price"])
        if buy_price <= 0:
            continue

        # How many pints can we buy?
        pints = int(BUDGET // buy_price)

        # Profit per pint = (0.9 * sell_price) - buy_price - freeze_cost
        realized_sell = (1.0 - DISCOUNT) * sell_price
        profit_per_pint = realized_sell - buy_price - FREEZE_COST
        total_profit = pints * profit_per_pint

        candidate = {
            "buy_month": str(buy_row["month"]),
            "sell_month": str(sell_row["month"]),
            "pints_purchased": pints,
            "expected_profit": total_profit
        }
        rows.append(candidate)

        if best is None or total_profit > best["expected_profit"]:
            best = candidate

timings = pd.DataFrame(rows).sort_values("expected_profit", ascending=False).reset_index(drop=True)
pd.DataFrame([best]).to_csv("timings.tsv", sep="\t", index=False)

print("Wrote timings.tsv")


Wrote timings.tsv


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [5]:
# YOUR CHANGES HERE

import pandas as pd

best = pd.read_csv("timings.tsv", sep="\t").iloc[0].to_dict()
buy_month = pd.Period(best["buy_month"], freq="M")
sell_month = pd.Period(best["sell_month"], freq="M")
pints = int(best["pints_purchased"])

forecast = pd.read_csv("strawberry-forecast.tsv", sep="\t")
forecast["month"] = pd.to_datetime(forecast["month"]).dt.to_period("M")

buy_price  = float(forecast.loc[forecast["month"] == buy_month,  "price"].iloc[0])
sell_price = float(forecast.loc[forecast["month"] == sell_month, "price"].iloc[0])

FREEZE_COST = 0.20
DISCOUNT = 0.10

# Baseline best profit
realized_sell = (1.0 - DISCOUNT) * sell_price
profit_per_pint = realized_sell - buy_price - FREEZE_COST
best_profit = pints * profit_per_pint

# Load backtest error std
err_stats = pd.read_csv("backtest-errors.tsv", sep="\t")
err_std = float(err_stats["std_error"].iloc[0])

# One-std-down sell price scenario
sell_price_down = sell_price - err_std
realized_sell_down = (1.0 - DISCOUNT) * sell_price_down
profit_per_pint_down = realized_sell_down - buy_price - FREEZE_COST
one_std_profit = pints * profit_per_pint_down

pd.DataFrame([{"best_profit": best_profit, "one_std_profit": one_std_profit}]) \
  .to_csv("check.tsv", sep="\t", index=False)

print("Wrote check.tsv")


Wrote check.tsv


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


In [1]:
from pathlib import Path

text = """Acknowledgement

Per Part 6, I am providing a brief written statement about my process and any tools used.
I used generative AI tools to understand the assignment requirements and to help structure
and implement the steps needed to create the required files. Specifically, I used generative
AI to understand and effectively make the backtest file (strawberry-backtest.tsv) and the
forecast file (strawberry-forecast.tsv). I reviewed and executed the code myself and accept
responsibility for the submitted work.
"""

Path("acknowledgement.txt").write_text(text, encoding="utf-8")
print("Wrote acknowledgement.txt")

# (Optional) show the file contents to confirm
print("\n--- acknowledgement.txt ---")
print(Path("acknowledgement.txt").read_text(encoding="utf-8"))

Wrote acknowledgement.txt

--- acknowledgement.txt ---
Acknowledgement

Per Part 6, I am providing a brief written statement about my process and any tools used.
I used generative AI tools to understand the assignment requirements and to help structure
and implement the steps needed to create the required files. Specifically, I used generative
AI to understand and effectively make the backtest file (strawberry-backtest.tsv) and the
forecast file (strawberry-forecast.tsv). I reviewed and executed the code myself and accept
responsibility for the submitted work.



Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.